In [1]:
import sys
from sys import getsizeof
import os

import time
from contextlib import contextmanager
import warnings

import numpy as np
import tqdm
from functools import reduce
from scipy.optimize import curve_fit

import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import pandas

from qiskit import *
from qiskit.qasm2 import dumps
from qiskit_aer import Aer

import math

In [2]:
%run ./Dependencies.ipynb

In [3]:
# Define the subdirectory name for storing figures
fig_subdirectory = "Figures"
if not os.path.exists(fig_subdirectory):    # Check if the subdirectory exists
    os.makedirs(fig_subdirectory)
    
# For writing data
data_subdirectory = "Data"
if not os.path.exists(data_subdirectory):
    os.makedirs(data_subdirectory)

# Setting the precision for floating points :
np.set_printoptions(precision=4)

In [4]:
write = False
eve_presence = 'Random'    #'Random'
ch_noise = 1e-4     # 0.000 - 0.300, 0.050 V/s QBER (eve detection)
eve_threshold = 20e-2    # QBER_threshold = eve_threshold - ch_noise ( 0.25(+-0.01) - 0.02 (+-0.001) = 0.23 +- 0.011 )

num_iter = 1
num_keys = 3
key_size = 1000

In [5]:
# %run ./initialization_bb84.ipynb
precision = eve_threshold/2 + ch_noise/2   
error_threshold = eve_threshold - ch_noise - precision

indices = np.arange(num_keys).astype('float32')
time_taken = np.zeros(num_keys, dtype = 'float32')    # avg time taken by a key of a certain length
QBERs = np.zeros(num_keys, dtype = 'float32')
KEY_RESERVOIR = []
KEY_RESERVOIR_len = np.zeros(num_keys, dtype = 'uint16')
Eve_detected = np.zeros(num_keys, dtype='uint8')

SKGR = 0
flag = 0
# Test()
init_vars = [num_iter, num_keys, key_size, write, eve_presence, ch_noise, eve_threshold, Eve_detected, KEY_RESERVOIR_len, time_taken, QBERs, KEY_RESERVOIR]
init_labels = "num_iter, num_keys, key_size, write, eve_presence, ch_noise, eve_threshold, Eve_detected, KEY_RESERVOIR_len, time_taken, QBERs, KEY_RESERVOIR".split(", ")

test(init_vars, init_labels)


Size and Type :

num_iter           :   Sys size : 28    ,  np size : NA      <class 'int'> 
num_keys           :   Sys size : 28    ,  np size : NA      <class 'int'> 
key_size           :   Sys size : 28    ,  np size : NA      <class 'int'> 
write              :   Sys size : 24    ,  np size : NA      <class 'bool'> 
eve_presence       :   Sys size : 55    ,  np size : NA      <class 'str'> 
ch_noise           :   Sys size : 24    ,  np size : NA      <class 'float'> 
eve_threshold      :   Sys size : 24    ,  np size : NA      <class 'float'> 
Eve_detected       :   Sys size : 115   ,  np size : 3      <class 'numpy.ndarray'> 
KEY_RESERVOIR_len  :   Sys size : 118   ,  np size : 6      <class 'numpy.ndarray'> 
time_taken         :   Sys size : 124   ,  np size : 12      <class 'numpy.ndarray'> 
QBERs              :   Sys size : 124   ,  np size : 12      <class 'numpy.ndarray'> 
KEY_RESERVOIR      :   Sys size : 56    ,  np size : NA      <class 'list'> 


Data Examples :
 
num_it

In [ ]:
START = time.time()
I = 0
while I < num_keys :
    
    with suppress_stdout():
        warnings.simplefilter('ignore')
        start = time.time()
        

In [ ]:
        try : 
            
            # %run ./Qiskit_rebuilt_4.ipynb 

            DATA_LENGTH = key_size    # In ./bb84_reservoir.ipynb
            KEY_LENGTH = int(DATA_LENGTH - np.ceil(np.log2(DATA_LENGTH)).astype(int) - 1)
            Unprocessed_key_len = 3*DATA_LENGTH
            
            # Preparation for encoding
            random.seed(0)    # Seed the random number generator. This will be used as our "coin flipper"
            
            # order = np.ceil(np.log2(Unprocessed_key_len)).astype(int)
            order = int(np.log2(Unprocessed_key_len))
            dim = int(2**(order/2))
            print(f"{Unprocessed_key_len = }, {KEY_LENGTH = }, {DATA_LENGTH = }, {order = }")
        
            alice_bits = generate_random_bits(Unprocessed_key_len)
            alice_bases = generate_random_bases(Unprocessed_key_len) # Alice randomly chooses a basis for each bit.
            
            print(f" Alice uncorrected {block(alice_bits, order)}")

            # Encode Alice's bits
            encoded_qubits = encode(alice_bits, alice_bases)

            if eve_presence == 'Random': eve = random.randint(0, 1)
            else: eve = int(eve_presence)
                
            label = 'Eve' if eve else 'Alice'
            print(label)

            qubits_received = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]    # Initializing the circuit
            errors_recorded = np.array([0, 0])    # Will keep track of the errors INJECTED deliberately by the algorithm
            
            if eve : 
                #print("Eve Present!")
                qubits_intercepted = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]
                
                errors_recorded = NoisyChannel(encoded_qubits, qubits_intercepted, 'Alice', errors_recorded, noise = ch_noise) ##Eve intercepts noisy states     
            
                eve_bases = generate_random_bases(Unprocessed_key_len) # Generate a random set of bases
                eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits
                
                # Eve encodes her decoy qubits and sends them along the quantum channel    
                encoded_intercepted_qubits = encode(eve_bits, eve_bases)    
                errors_recorded = NoisyChannel(encoded_intercepted_qubits, qubits_received, 'Eve', errors_recorded, noise = ch_noise) ## Eve sends noisy states to Bob
            
            else : 
                errors_recorded = NoisyChannel(encoded_qubits, qubits_received, 'Alice', errors_recorded, noise = ch_noise) ## Alice sends noisy states to Bob
            
            bob_bases = generate_random_bases(Unprocessed_key_len) # Bob randomly chooses a basis for each bit.
            
            # Measurement
            bob_bits = measure(qubits_received, bob_bases)
            
            # print(f" {type(bob_bits[0]) = },  {type(alice_bits[0]) = }")

            BROADCAST = alice_bases    # Alice tells Bob which bases she used. BROADCAST uses classical channel
            
            # Store the indices of the bases they share in common
            common_bases = [i for i in range(Unprocessed_key_len) if bob_bases[i] == BROADCAST[i]]
            bob_bits = [bob_bits[index] for index in common_bases]
            bob_bits = ''.join(bob_bits)
            
            print(f"\nAlice sent {block(alice_bits, order)} \n\nBob measured {block(bob_bits, order)}")
            
            BROADCAST = common_bases    # Bob tells Alice which bases they shared in common
            
            alice_bits = [alice_bits[index] for index in BROADCAST]    # Alice keeps only the bits they shared in common
            alice_bits = ''.join(alice_bits)


            sample = len(alice_bits)//3    # len(alice_bits) >= 3
            errors_detected = 0
            
            for _ in range(sample):
                bit_index = random.randrange(len(alice_bits)) 
                
                if alice_bits[bit_index] != bob_bits[bit_index]:  errors_detected += 1    #calculating errors
                    
                alice_bits = alice_bits[:bit_index] + alice_bits[bit_index+1 :] #removing tested bits from key strings
                bob_bits = bob_bits[:bit_index] + bob_bits[bit_index+1 :]
            
            # order = np.ceil(np.log2(len(alice_bits))).astype(int)
            order = Order(alice_bits)

            print(f' Errors inflicted[bit, phase] : {errors_recorded}, Errors detected(total) : {errors_detected}, {sample = }')

            
            ### QBER should be ~ 0.5 (instead of ~0.25) in presence of Eve, because the sample size is 1/3 of the bits AFTER sifting.
            
            QBER = round(errors_detected/sample, 5) # calculating QBER and saving the answer to two decimal places
            print(f"{QBER = }")
            
            flag = 0
            
            print(f"\n Error Threshold : {error_threshold}")
            
            if QBER > error_threshold:
                num_keys += 1
                key = ''
                I -= 1
                flag = 0
                raise RuntimeError('\n Eve{} detected'.format('' if eve else ' Falsely'))
                # print(""" Key not secure. Aborting protocol...
                # \r NO NEED FOR PROCEEDING TO ERROR CORRECTION \n\n\n\n""".format('' if eve else ' Falsely'))
                # elif eve and eve_presence : input(" Stuck in infinite loop ")
            
            else :
                print(" Key is secure")
                flag = 1
                if eve : print(' Eve went unnoticed : ', end = " ")
                else : print(' Eve not present : ', end = " ")
            
                # if not flag : print(f" \n NO NEED FOR PROCEEDING TO ERROR CORRECTION \n\n\n\n")
            
                # KEY_RESERVOIR = np.concatenate(KEY_RESERVOIR, alice_bits[KEY_LENGTH:]


        except Warning: print('Eve Detected')

In [ ]:
        if flag == 1:
            # %run ./error_correction.ipynb
            # After Sifting
            PARITY_DICT, _ = parity(order)   # Returns empty PARITY_DICT, bin_rep
            
            alice_block = create_parity_block(alice_bits, order, PARITY_DICT)    # Encodes parity on the block
            
            BROADCAST = PARITY_DICT
            
            bob_block = create_parity_block(bob_bits, order, BROADCAST)
            ### Both blocks have been created, now the hamming protocol can be applied
            
            err_count, loc, binary_rep = hamming(bob_block, order)

            print(f"Hamming error counts : {err_count} loc : {loc} ({binary_rep})")
            total_errors_recorded = errors_recorded[0] + errors_recorded[1]
            
            if err_count != 0 :
                try : 
                    if err_count == 1: 
                        bob_bits[loc] = np.mod(bob_bits[loc] + 1, 2)
                        alice_bits = bob_bits
                        print("\n Errors corrected")
            
                    else: print("\n More than one errors present")
            
                except :
                    raise KeyError('Location Invalid')

            # Still need to remove the parity bits
            key = ""
            for bit in alice_bits:    # Or bob_bits, since both should be the same
                key += bit

In [ ]:
        
    time_taken[I] = round(time.time() - start, 4)
    
    QBERs[I] = QBER
    KEY_RESERVOIR.append(key)
    KEY_RESERVOIR_len[I] = len(key)

    Eve_detected[I] = ((QBER >= error_threshold) and eve) + ((QBER < error_threshold) and not eve)    # Whether or not the DETECTION of Eve is CORRECT

    prog_interval = int((I+1)*100/num_keys)
    print(f"\r Keys generated: [{'='*(prog_interval)}>{' '*(num_keys-prog_interval)}] {I+1}/{num_keys}", end = '')
    I += 1

total_time = time.time() - START
print("\n", total_time)

In [ ]:
print(total_time)
elapsed_time = f"{str(int(total_time//3600))}h  {str( int((total_time%3600)//60) )}m  {str(int(total_time%60))}s"
elapsed_time

In [ ]:
SKGR = len(KEY_RESERVOIR_len)/total_time
SKGR

In [ ]:
# Open the files to write data to
title = f"\nKEY RESERVOIR[Noise {ch_noise}][Eve {eve_presence}][Eve Threshold {eve_threshold}] "
simulation_parameters = f"\nNumber of keys : {num_keys},   Aimed length of keys : {key_size}  \nEve Presence : {eve_presence},   Channel Noise : {ch_noise},   Eve's threshold : {eve_threshold},   Precision of Error Measurement : {precision},   Error Threshold : {error_threshold} \nTime Taken : {elapsed_time}\n"

avg_time_taken = sum([time_taken[I] for I in range(num_keys)])/num_keys   
avg_out_len = sum([KEY_RESERVOIR_len[I] for I in range(num_keys)])/num_keys
avg_QBERs = sum([QBERs[I] for I in range(num_keys)])/num_keys

In [ ]:
incorrect = len(Eve_detected) - sum(Eve_detected)
detection = f"Eve detection was incorrect : {incorrect}/{sum(Eve_detected)} times (= {incorrect/sum(Eve_detected)} ); for QBER Threshold : {error_threshold}"
print(detection)
# print(max(sum(Eve_detected))) 
print(f'Eve Detection[Noise {ch_noise}]')

plt.figure(figsize=(12, 5))

# plt.xlabel(f'Key Index')
# plt.ylabel(f'Time Taken')
# # plt.ylabel(f'Times correctly detected Eve(out of {num_keys} keys)')
# xticks = indices[::int(num_keys/10)]
# plt.xticks(xticks)
# plt.yticks(np.arange(max(time_taken)+1)[::5])

# plt.minorticks_on()
# plt.grid(True)
# plt.tight_layout()

# plt.scatter(indices, Eve_detected)
df = pandas.DataFrame({'indices' : indices, 'time_taken' : time_taken, 'Eve_detected' : Eve_detected}, index = indices )
# sns.lmplot(x = 'indices', y = 'time_taken', hue = 'Eve_detected', data = df)
sns.scatterplot(x = indices, y = time_taken, hue = Eve_detected)
# print(f"{xticks = }")

In [ ]:
var = [num_keys, key_size, write, eve_presence, ch_noise, eve_threshold, Eve_detected, KEY_RESERVOIR_len, time_taken, QBERs, KEY_RESERVOIR]
labels = "num_keys, key_size, write, eve_presence, ch_noise, eve_threshold, Eve_detected, KEY_RESERVOIR_len, time_taken, QBERs, KEY_RESERVOIR".split(", ")
    
test(var, labels)

In [ ]:
False_detections = np.where(Eve_detected == 0)    #[(i, j) for (i, j) in zip(np.where(Eve_detected == 0)[0], np.where(Eve_detected == 0)[1])]
try :
    print(False_detections[:9], '...')
except :
    print(False_detections[0])
finally :
    print(f"{len(False_detections) = }")

In [ ]:
# Cast indices and QBERs to float32 or float64
indices = np.array(indices, dtype=np.float32)
QBERs = np.array(QBERs, dtype=np.float32)

fig, ax = plt.subplots(2, 2, figsize=(12, 8))
# For key length
coefficients_len = np.polyfit(indices, KEY_RESERVOIR_len, 1)
polynomial_len = np.poly1d(coefficients_len)
equation_len = f'y = {coefficients_len[0]:.2f}x + {coefficients_len[1]:.2f}'
ax[0, 0].text(0.05, 0.95, equation_len, transform=ax[0, 0].transAxes, fontsize=10, verticalalignment='top')
ax[0, 0].set_xlabel('Indices')
ax[0, 0].set_ylabel('Length of final key after Information Reconciliation')
ax[0, 0].set_title("Key length")
ax[0, 0].plot(indices, KEY_RESERVOIR_len)
ax[0, 0].plot(indices, polynomial_len(indices), linestyle='--')
ax[0, 0].minorticks_on()
ax[0, 0].grid(True)

# For QBER
coefficients_qber = np.polyfit(indices, QBERs, 1)
polynomial_qber = np.poly1d(coefficients_qber)
equation_qber = f'y = {coefficients_qber[0]:.2f}x + {coefficients_qber[1]:.2f}'
ax[0, 1].text(0.05, 0.95, equation_qber, transform=ax[0, 1].transAxes, fontsize=10, verticalalignment='top')
ax[0, 1].set_xlabel('Indices')
ax[0, 1].set_ylabel('Average QBER')
ax[0, 1].set_title("QBER")
ax[0, 1].plot(indices, QBERs)
ax[0, 1].plot(indices, polynomial_qber(indices), linestyle='--')
ax[0, 1].minorticks_on()
ax[0, 1].grid(True)

# For time taken for 1 cycle
coefficients_tt = np.polyfit(indices, time_taken, 1) #sns.lineplot(np.arange(num_keys), time_taken, linestyle = '--')
polynomial_tt = np.poly1d(coefficients_tt)
equation_tt = f'y = {coefficients_tt[0]:.2f}x + {coefficients_tt[1]:.2f}'
ax[1, 0].text(0.05, 0.95, equation_tt, transform=ax[1, 0].transAxes, fontsize=10, verticalalignment='top')
ax[1, 0].set_xlabel('Indices')
ax[1, 0].set_ylabel('Time taken(in seconds) for 1 cycle')
ax[1, 0].set_title("Time taken")
ax[1, 0].plot(indices, time_taken)
ax[1, 0].plot(indices, polynomial_tt(indices), linestyle='--')
ax[1, 0].minorticks_on()
ax[1, 0].grid(True)

# Fit the data to the hyperbolic function
# popt, pcov = curve_fit(hyperbolic_fit, indices, SKGR)
# a, b = popt
# # Generate the polynomial using the fitted parameters
# fitted_SKGR = hyperbolic_fit(indices, *popt)
# equation_SKGR = f'y = {a:.2f}/x + {b:.2f}'
# ax[1, 1].text(0.05, 0.95, equation_SKGR, transform=ax[1, 1].transAxes, fontsize=10, verticalalignment='top')
# ax[1, 1].set_xlabel('Indices of initial key')
# ax[1, 1].set_ylabel('Secure key generation rate')
# ax[1, 1].set_title("Input key indices vs SKGR")
# ax[1, 1].plot(indices, SKGR, 'o', label='Data')  # Plot the original data
# ax[1, 1].plot(indices, fitted_SKGR, linestyle='--', label='Hyperbolic fit')  # Plot the hyperbolic fit
# ax[1, 1].minorticks_on()
# ax[1, 1].grid(True)


# Set the super title for the entire figure
plt.suptitle(title, weight='bold')

# Display the plots
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Print the title
print(title)


In [ ]:
if write_bb84 :
    filename = f"Data.txt"
    key_strings = f"Keys.txt"
    data_path = os.path.join(data_subdirectory, filename)
    key_path = os.path.join(data_subdirectory, key_strings)
    file = open(data_path, "a")
    file2 = open(key_path, "a")
    
    file.write(simulation_parameters)
    file2.write(simulation_parameters)

    
    file.write(f"\nInput Length = [{', '.join(map(str, in_len))}] \nAverage Time Taken = [{', '.join(map(str, avg_time_taken))}] \nAverage QBER = [{', '.join(map(str, avg_QBERs))}] \nAverage Output length = [{', '.join(map(str, avg_out_len))}]\n \nSKGR = [{', '.join(map(str, SKGR))}]")
    file.close()
    
    file2.write(detection)
    file2.write(f" False detection indices : {False_detections}")
    file2.write(f"\nKeys = [{', '.join(map(str, keys))}] \n\n Eve detection = [{', '.join(map(str, Eve_detected))}]")
    file2.close()

    write = False
    print("Files updated")

In [ ]:
plt.plot(indices, KEY_RESERVOIR_len/1000, label = "Key length/1000")
plt.plot(indices, QBERs, label = "QBERs")
plt.plot(indices, time_taken/100, label = "time_taken/100")
# plt.plot(indices, SKGR, label = 'SKGR')

plt.xlabel('index')
plt.ylabel('counts')
plt.title(f'All parameters in one for {title}')
plt.legend()

print(f'All_parameters[Noise {ch_noise}]')